In [26]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


In [27]:
# To run this, you can install BeautifulSoup
# https://pypi.python.org/pypi/beautifulsoup4

# Or download the file
# http://beautiful-soup-4
# and unzip it in the same directory as this file
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [28]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [29]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')


soup ready


In [30]:
table = soup.find('table',{'class':'wikitable sortable'})
#table

In [31]:
table_rows = table.find_all('tr')

#table_rows

In [60]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

print(df.head(5))

  PostalCode           Borough     Neighbourhood
1        M1A      Not assigned      Not assigned
2        M2A      Not assigned      Not assigned
3        M3A        North York         Parkwoods
4        M4A        North York  Victoria Village
5        M5A  Downtown Toronto      Harbourfront


## Data Transformed into DataFrame

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode       287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [34]:
df.shape

(287, 3)

In [61]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


In [73]:
df1 = df.reset_index()
df1.head()
df1.columns
df1.drop('index', axis=1, inplace=True)

Index(['index', 'PostalCode', 'Borough', 'Neighbourhood'], dtype='object')

In [81]:
df1.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


## Question 2 
(using the Geospatial_Coordiante.csv)


In [83]:
!pip install geopy

In [85]:
data2 = pd.read_csv("https://cocl.us/Geospatial_data") 
# Preview the first 5 lines of the loaded data 
data2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [86]:
data2.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
#data2.head()

In [87]:
data = pd.merge(df1, data2, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497


## Question 3
analyzing Borough

In [104]:
CLIENT_ID = 'VO3CR5T34VIYBYGSOK4QWJ30HTRNX4N3YLRODMXLYK4CRWTH' # your Foursquare ID
CLIENT_SECRET = 'INL4034YUC0HEUYMEV40OAHSSOIMFOHZO5XWDNS5I3YVSAHX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VO3CR5T34VIYBYGSOK4QWJ30HTRNX4N3YLRODMXLYK4CRWTH
CLIENT_SECRET:INL4034YUC0HEUYMEV40OAHSSOIMFOHZO5XWDNS5I3YVSAHX


In [89]:
data.loc[0, 'Borough']

'Scarborough'

In [91]:
Borough_latitude = data.loc[0, 'Latitude'] # Borough latitude value
Borough_longitude = data.loc[0, 'Longitude'] # Borough longitude value

Borough_name = data.loc[0, 'Borough'] # Borough name

print('Latitude and longitude values of {} are {}, {}.'.format(Borough_name, 
                                                               Borough_latitude, 
                                                               Borough_longitude))



Latitude and longitude values of Scarborough are 43.806686299999996, -79.19435340000001.


In [115]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Borough_latitude, 
    Borough_longitude, 
    radius, 
    LIMIT)
url # display UR

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e512fad1a4b0a001b3e523e'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 43.806704300018,
    'lng': -79.19432850472293},
   'sw': {'lat': 43.80666829998199, 'lng': -79.19437829527709}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [103]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

KeyError: 'groups'

In [110]:
# function that extracts the category of the town
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [119]:

Borough_venues = getNearbyVenues(names=data['Borough'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )
print(Borough_venues.shape)
Borough_venues.head()

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central T

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
1,Scarborough,43.799525,-79.318389,Price Chopper,43.799445,-79.318563,Grocery Store
2,North York,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
3,Central Toronto,43.704324,-79.388790,Jules Cafe Patisserie,43.704138,-79.388413,Dessert Shop
4,Central Toronto,43.704324,-79.388790,Thobors Boulangerie Patisserie Café,43.704514,-79.388616,Café


In [121]:
Borough_grouped = Borough_venues.groupby('Borough').count()

In [124]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Borough_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:971: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  return_n_iter=True)


array([2, 1, 0, 3, 0, 2], dtype=int32)

In [126]:
# add clustering labels
Borough_venues.insert(0, 'Cluster Labels', kmeans.labels_)



# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
data_merged = data.join(Borough_venues.set_index('Borough'), on='Borough')

data_merged.head() # check the last columns!

ValueError: Length of values does not match length of index

In [150]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location=[Borough Latitude, Borough longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


map_clusters



SyntaxError: invalid syntax (<ipython-input-150-1c9520ecaf96>, line 7)

In [148]:
data['Latitude']

0      43.806686
1      43.806686
2      43.784535
3      43.784535
4      43.784535
5      43.763573
6      43.763573
7      43.763573
8      43.770992
9      43.773136
10     43.744734
11     43.727929
12     43.727929
13     43.727929
14     43.711112
15     43.711112
16     43.711112
17     43.716316
18     43.716316
19     43.716316
20     43.692657
21     43.692657
22     43.757410
23     43.757410
24     43.757410
25     43.750072
26     43.750072
27     43.794200
28     43.781638
29     43.781638
         ...    
180    43.628841
181    43.628841
182    43.667856
183    43.650943
184    43.650943
185    43.650943
186    43.650943
187    43.650943
188    43.643515
189    43.643515
190    43.643515
191    43.643515
192    43.756303
193    43.724766
194    43.724766
195    43.706876
196    43.696319
197    43.688905
198    43.688905
199    43.688905
200    43.688905
201    43.739416
202    43.739416
203    43.739416
204    43.739416
205    43.739416
206    43.739416
207    43.7394

In [ ]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(data['Latitude'], data['Longitude'], data['Borough']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       